In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from fedbiomed.researcher.environ import TMP_DIR
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=TMP_DIR+'/')
model_file = tmp_dir_model.name + '/fedbiosklearn.py'

Customize params is an optional method needed when there is type difference between fedbiomed and the scikit learn 
framework. i.e fedbiomed uses json serialization to send list objects and scikit learn SGD Regressor parameters
are of type numpy array.

In [3]:
%%writefile "$model_file"

from fedbiomed.common.fedbiosklearn import SkLearnModel
from sklearn.linear_model import SGDRegressor


class SGDRegressorTrainingPlan(SkLearnModel):
    #TODO impact of kwargs ?, is it optional ????
    def __init__(self, kwargs):
        super(SGDRegressorTrainingPlan, self).__init__(kwargs)
    
    def after_training_params(self):
        return {'coef_':  self.reg.coef_  , 'intercept_': self.reg.intercept_}
    
    def training_step(self,X,y):
        self.reg.partial_fit(X,y)
        print('After training coef ', self.reg.coef_ , 'Intercept ',self.reg.intercept_ )
    
    def training_data(self,batch_size = 50):
        dataset = pd.read_csv(self.dataset_path,header=None,delimiter=',')
        X = dataset.iloc[:,0:5].values
        print('Rendering X ' , X , ' type ', type(X))
        y = dataset.iloc[:,4]
        print('Rendering y ' , y , ' type ', type(y.values))
        return (X,y.values)
    

Writing /user/jsaray/home/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/tmpsy9v_yki/fedbiosklearn.py


In [4]:
model_args = { 'max_iter':1000, 'tol': 1e-3  }

training_args = {
    'batch_size': 50, 
    'lr': 1e-3, 
    'epochs': 5, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

In [5]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['skid']
rounds = 5

exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='SGDRegressorTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 client_selection_strategy=None)

Messaging researcher_fe36f64f-19e5-4500-87c4-31604ef82c2f connected with result code 0
Searching for clients with data tags: ['skid'] ...
2021-08-16 17:41:23.636770 [ RESEARCHER ] message received. {'researcher_id': 'researcher_fe36f64f-19e5-4500-87c4-31604ef82c2f', 'success': True, 'databases': [{'name': 'skid', 'data_type': 'csv', 'tags': ['skid'], 'description': 'skid', 'shape': [49, 5], 'dataset_id': 'dataset_ab47a8e2-cb45-4312-a01f-f61332439724'}], 'count': 1, 'client_id': 'client_d43ec19d-4af1-4b83-9f07-301afdf280fb', 'command': 'search'}
2021-08-16 17:41:23.637488 [ RESEARCHER ] message received. {'researcher_id': 'researcher_fe36f64f-19e5-4500-87c4-31604ef82c2f', 'success': True, 'databases': [{'name': 'skid', 'data_type': 'csv', 'tags': ['skid'], 'description': 'skid', 'shape': [49, 5], 'dataset_id': 'dataset_7759ce82-72b2-4a66-9f78-0d2a65ceb231'}], 'count': 1, 'client_id': 'client_bd048510-59be-4861-9ab4-056ba17da12c', 'command': 'search'}
saving without any params 


In [6]:
exp.run()

Sampled clients in round  0   ['client_d43ec19d-4af1-4b83-9f07-301afdf280fb', 'client_bd048510-59be-4861-9ab4-056ba17da12c']
[ RESEARCHER ] Send message to client  client_d43ec19d-4af1-4b83-9f07-301afdf280fb {'researcher_id': 'researcher_fe36f64f-19e5-4500-87c4-31604ef82c2f', 'job_id': '64f96702-5d9d-4dd5-b593-9019b5cd14eb', 'training_args': {'batch_size': 50, 'lr': 0.001, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {'max_iter': 1000, 'tol': 0.001}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/08/16/my_model_tQTZnHR.py', 'params_url': 'http://localhost:8844/media/uploads/2021/08/16/my_model_ZZTjwKl.pt', 'model_class': 'SGDRegressorTrainingPlan', 'training_data': {'client_d43ec19d-4af1-4b83-9f07-301afdf280fb': ['dataset_ab47a8e2-cb45-4312-a01f-f61332439724']}}
researcher_fe36f64f-19e5-4500-87c4-31604ef82c2f
[ RESEARCHER ] Send message to client  client_bd048510-59be-4861-9ab4-056ba17da12c {'researcher_id': 'researcher_fe36f64f-19e5-45

filename is  /user/jsaray/home/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/my_model_907bc26c-8977-4f80-aea1-3012e9302b60.pt
Loaded coef - parms COEF (JOB)   with parameters coef  [0.02890497 0.07215293 0.02456299 0.0482583  0.64412401]  intercept_  [0.1341067]
Clients that successfully reply in round  1   ['client_d43ec19d-4af1-4b83-9f07-301afdf280fb', 'client_bd048510-59be-4861-9ab4-056ba17da12c']
Model params  {'coef_': array([0.02890533, 0.0721023 , 0.02447643, 0.04780634, 0.64429452]), 'intercept_': array([0.13376697])}
=====Weights====== tensor([0.5000, 0.5000])  model_params  [{'coef_': array([0.02890533, 0.0721023 , 0.02447643, 0.04780634, 0.64429452]), 'intercept_': array([0.13376697])}, {'coef_': array([0.02890497, 0.07215293, 0.02456299, 0.0482583 , 0.64412401]), 'intercept_': array([0.1341067])}]  proportions   [tensor(0.5000), tensor(0.5000)]  avg params  {'coef_': array([0., 0., 0., 0., 0.]), 'intercept_': array([0.])}
 =============avg params=======

2021-08-16 17:43:04.121294 [ RESEARCHER ] message received. {'researcher_id': 'researcher_fe36f64f-19e5-4500-87c4-31604ef82c2f', 'job_id': '64f96702-5d9d-4dd5-b593-9019b5cd14eb', 'success': True, 'client_id': 'client_d43ec19d-4af1-4b83-9f07-301afdf280fb', 'dataset_id': 'dataset_ab47a8e2-cb45-4312-a01f-f61332439724', 'params_url': 'http://localhost:8844/media/uploads/2021/08/16/node_params_6f8eddae-8503-49a7-b299-4922e535e701.pt', 'timing': {'rtime_training': 0.006364235003275098, 'ptime_training': 0.0063086039999999954}, 'msg': '', 'command': 'train'}
2021-08-16 17:43:05.071970 [ RESEARCHER ] message received. {'researcher_id': 'researcher_fe36f64f-19e5-4500-87c4-31604ef82c2f', 'job_id': '64f96702-5d9d-4dd5-b593-9019b5cd14eb', 'success': True, 'client_id': 'client_bd048510-59be-4861-9ab4-056ba17da12c', 'dataset_id': 'dataset_7759ce82-72b2-4a66-9f78-0d2a65ceb231', 'params_url': 'http://localhost:8844/media/uploads/2021/08/16/node_params_f5da2f04-5daf-413e-b350-5a89501c34b2.pt', 'timing'

In [7]:
n_samples, n_features = 100, 5
testing_samples = 40
rng = np.random.RandomState(1)
A = np.array([[5],
       [8],
       [9],
       [5],
       [0]])
def test_data():
    X_test = rng.randn(testing_samples, n_features).reshape([testing_samples, n_features])
    y_test = X_test.dot(A) + rng.randn(testing_samples).reshape([testing_samples,1])
    return X_test, y_test

In [8]:
from sklearn.linear_model import SGDRegressor

In [9]:
fed_model = SGDRegressor(max_iter=1000, tol=1e-3)
X_test, y_test = test_data()

In [13]:
X_test.shape

(40, 5)

In [14]:
y_test.shape

(40, 1)

In [16]:
exp._aggregated_params[0]['params']

{'coef_': array([-0.02762315,  0.03511984, -0.00151897,  0.0407575 ,  0.344748  ]),
 'intercept_': array([0.11286286])}

In [17]:
exp._aggregated_params[1]['params']

{'coef_': array([-0.03290315,  0.04135906, -0.0036068 ,  0.04225345,  0.50920882]),
 'intercept_': array([0.13503038])}

In [19]:
exp._aggregated_params[2]['params']

{'coef_': array([-0.03285988,  0.04227644, -0.00573254,  0.03831777,  0.61894459]),
 'intercept_': array([0.13432315])}

In [20]:
exp._aggregated_params[3]['params']

{'coef_': array([-0.03110757,  0.03885024, -0.00546323,  0.03020861,  0.69346249]),
 'intercept_': array([0.12715013])}

In [21]:
exp._aggregated_params[4]['params']

{'coef_': array([-0.03516372,  0.02830532,  0.00555754,  0.00993721,  0.77441406]),
 'intercept_': array([0.09274101])}

In [17]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [10]:
testing_error = []

for i in range(rounds):
    fed_model.coef_ = exp._aggregated_params[i]['params']['coef_']
    fed_model.intercept_ = exp._aggregated_params[i]['params']['intercept_']
    
    mse = np.mean((fed_model.predict(X_test).ravel() - y_test.ravel())**2)
    print('MSE ', mse)
    testing_error.append(mse)

MSE  177.38354527510978
MSE  178.1822791029444
MSE  178.97386606885615
MSE  179.64031077910417
MSE  180.17794175010937


In [26]:
testing_error

[178.03450216971504,
 178.68256697912727,
 179.29662378991694,
 179.83256997069486,
 180.54893347282993]

In [17]:
plt.plot(testing_error, label = 'federated test error')
plt.hlines(np.mean((X_test.dot(A).ravel() - y_test.ravel())**2), 0, N_rounds-1, color = 'red', alpha = 0.7, label = 'ground truth')
plt.xlabel('Round')
plt.legend()

NameError: name 'plt' is not defined

In [10]:
exp._aggregated_params[0]['params']['coef_']

array([-0.08835477, -0.02177115,  0.09723794,  0.03879841,  0.00388391])

In [11]:
exp._aggregated_params[1]['params']['coef_']

array([-0.0857399 , -0.02418705,  0.09421684,  0.04130261,  0.00110738])

In [12]:
exp._aggregated_params[2]['params']['coef_']

array([-0.08628797, -0.02662039,  0.09914285,  0.04398299,  0.00211929])

In [13]:
exp._aggregated_params[3]['params']['coef_']

array([-0.0876811 , -0.02231482,  0.09693187,  0.04262923,  0.00136258])

In [ ]:
exp._aggregated_params[1]['params']['coef_']

In [ ]:
exp._aggregated_params[1]['params']['coef_']

In [8]:
exp._aggregated_params[1]

{'params': {'coef_': array([-0.09106583, -0.02673528,  0.09718337,  0.04359848, -0.00150982]),
  'intercept_': array([0.02567093])},
 'params_path': '/home/jsaray/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/researcher_params_b98dd684-2020-4337-a061-0e1f7989c33d.pt'}

In [9]:
exp._aggregated_params[2]

{'params': {'coef_': array([-0.08315385, -0.02271357,  0.09840193,  0.04169297,  0.00091099]),
  'intercept_': array([0.02895132])},
 'params_path': '/home/jsaray/INRIA-PROJECTS/features/sklearn_integ_refactored/fedbiomed/var/tmp/researcher_params_e4a565be-dfe9-42a3-8d45-96915f403bd7.pt'}

In [ ]:
%%writefile "$model_file"

from fedbiomed.common.fedbiosklearn import SkLearnModel
from sklearn.linear_model import SGDRegressor


class SGDRegressorTrainingPlan(SkLearnModel):
    #TODO impact of kwargs ?, is it optional ????
    def __init__(self, kwargs):
        super(SGDRegressorTrainingPlan, self).__init__()
        self.reg = SGDRegressor(max_iter=kwargs['max_iter'], tol=kwargs['tol'])  
        if(kwargs.get('coef_') is not None and kwargs.get('intercept_') is not None):
            if isinstance(kwargs['coef_'],list) and isinstance(kwargs['intercept_'],list):
                self.reg.coef_ = np.array(kwargs['coef_'])
                self.reg.intercept_ = np.array(kwargs['intercept_'])
    
    
    def customize_params(self,kwargs):
        
    
    def after_training_params(self):
        return {'coef_':  self.reg.coef_  , 'intercept_': self.reg.intercept_}
    
    def training_step(self,X,y):
        self.reg.partial_fit(X,y)
        print('After training coef ', self.reg.coef_ , 'Intercept ',self.reg.intercept_ )
    
    def training_data(self,batch_size = 48):
        n_samples, n_features = 100, 5
        testing_samples = 40

        ### Creating a random dataset
        rng = np.random.RandomState(1)

        y = []
        X = []

        ### Creating a random linear transformation
        A = rng.randint(0,10,size = n_features).reshape([n_features,1])

        ### For every center we create random X and y dataset with same generative rule (the matrix A)

        X = rng.randn(n_samples, n_features)
        y = rng.randn(n_samples)
        print('X shape' , X.shape , 'y shape', y.shape)
        return (X,y)